In [1]:
import pandas as pd
from collections import Counter
import functools
import tqdm
import re
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import shuffle
import csv

In [4]:
N_TOTAL = 119398

N_TRAIN =  90000
N_TEST = N_TOTAL - N_TRAIN

In [5]:
# df = pd.read_csv(r"C:\dataA\features.csv") # Either run this cell or the next one with a pickle

In [6]:
import pickle, gzip
df = pickle.load(gzip.open("/media/artiom/Terminator/NLP-contest/taskA/dftrain_base_features.pickle", "rb" ))

In [7]:
df = shuffle(df)

In [8]:
df_train = df.loc[:N_TRAIN]
df_test = df.loc[N_TRAIN:]

In [13]:
columns = ['len_paragraph', 'len_question', 'len_intersection', 'idf_question', 'idf_paragraph', 'idf_intersection']
model = GradientBoostingClassifier().fit(df_train[columns], df_train['target'])

In [10]:
# Другой вариант обучения - с меньшим количеством колонок
columns = ['len_paragraph', 'len_question']
model = GradientBoostingClassifier().fit(df_train[columns], df_train['target'])

In [14]:
df_test['prediction'] = model.predict_proba(df_test[columns])[:, 1]

/home/artiom/ai.ml/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
pos = 0
correctly_guessed_zeros = 0
correctly_guessed_ones = 0

predictor_said_yes = 0

ones_in_dataset = 0

for index, row in tqdm.tqdm(df_test.iterrows(), total=df_test.shape[0]):
    target = row["target"]
    prediction = row["prediction"]
    
    ones_in_dataset += target
    predictor_said_yes += prediction
                 
    if round(prediction) == target:
        pos += 1
        
        if target == 0:
            correctly_guessed_zeros += 1
        elif target == 1:
            correctly_guessed_ones += 1
        else:
            print("Wow!")
            
            
zeros_in_dataset = df_test.shape[0] - ones_in_dataset
            
print("Длина тестовой части", df_test.shape[0])            
print("Всего в датасете единиц: ", ones_in_dataset)        
print("Всего в датасете нулей: ", df_test.shape[0] - ones_in_dataset)      
print("ДОЛЯ ВЕРНЫХ ОТВЕТОВ:", pos / df_test.shape[0])
print("Верно угадано единиц: ",correctly_guessed_ones," (доля ", correctly_guessed_ones/ones_in_dataset,")")
print("Верно угадано нулей: ", correctly_guessed_zeros," (доля ", correctly_guessed_zeros/zeros_in_dataset,")")
print("Предиктор сказал 'Да' ",predictor_said_yes, "раз доля ", predictor_said_yes/df_test.shape[0],")")

100%|██████████| 31406/31406 [00:05<00:00, 5368.90it/s]

Длина тестовой части 31406
Всего в датасете единиц:  9045.0
Всего в датасете нулей:  22361.0
ДОЛЯ ВЕРНЫХ ОТВЕТОВ: 0.9736674520792206
Верно угадано единиц:  8621  (доля  0.9531232725262576 )
Верно угадано нулей:  21958  (доля  0.9819775501990072 )
Предиктор сказал 'Да'  9000.21108920702 раз доля  0.28657616663080365 )


In [19]:
9518/23700

0.40160337552742614

In [31]:
df_test.shape[0]

1934